In [17]:
import pandas as pd
import numpy as np
import os
import sys
import scanpy as sc
import anndata as ad
import scipy
import time
import gget
from sklearn.decomposition import PCA

import plotly
import plotly.express as px
import plotly.offline as go_offline
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

# local 
import utils as ut

In [2]:
fpath = "/nfs/turbo/umms-indikar/shared/projects/MC3R/processed_data/all_subclustered.h5ad"
adata = sc.read(fpath)
adata

AnnData object with n_obs × n_vars = 33647 × 25060
    obs: 'batch', 'reference_embedding_C7_named', 'reference_embedding_C25_named', 'C25_named', 'C7_named', 'reference_embedding_C7_named_clean', 'reference_embedding_C25_named_clean', 'C25_named_clean', 'C7_named_clean', 'UMAP1', 'UMAP2', 'Diet', 'Sex', 'broad_type', 'n_genes', 'color', 'leiden', 'cluster_num', 'Cluster', 'cluster_cat', 'U1', 'U2', 'cell_type', 'NU1', 'NU2', 'neuron_cluster_num', 'Nueron_Cluster', 'neuron_cluster_cat', 'HypoMap Predicitions', 'HypoMap Subclusters', 'neuron_subtype', 'batch_name'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Cluster', 'Diet_colors', 'HypoMap Predicitions_colors', 'HypoMap Subclusters_colors', 'Nueron_Cluster', 'Nueron_Cluster_colors', 'Sex_colors', 'batch_colors', 'dendrogram_Nueron_Cluster', 'hvg', 'leiden', 'log1p', 'neighbors', 'neuron_subtype_colors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obs

In [3]:
alpha = 0.05
deg = sc.get.rank_genes_groups_df(adata, 
                                  group=None,
                                  key='Nueron_Cluster')

deg = deg.sort_values(by=['group', 'logfoldchanges'],
                     ascending=[True, False])

print(f"{deg.shape}")

# significance thresholding
deg = deg[deg['pvals_adj'] <= alpha]

# rename columns
deg.columns = ['cluster', 'gene_symbol', 'scores', 'log2foldchanges', 'pvals', 'pvals_benjamini-hochberg']

# filter out some genes
deg = deg[~deg['gene_symbol'].str.startswith('Gm')]
deg = deg[~deg['gene_symbol'].str.startswith('MT-')]
deg = deg[~deg['gene_symbol'].str.endswith('Rik')]
print(f"{deg.shape}")


deg.head()

(300720, 6)
(67010, 6)


,cluster,gene_symbol,scores,log2foldchanges,pvals,pvals_benjamini-hochberg
23,C1,Dlx6os1,9.485873,0.944240,2.403620e-21,1.594355e-20
73,C1,Meis2,3.491885,0.878733,4.796238e-04,1.100574e-03
7,C1,Gad2,12.999899,0.695774,1.225052e-38,3.561462e-37
2,C1,Nrxn3,25.884439,0.685811,9.970003e-148,2.776092e-144
94,C1,Dlx1,2.713853,0.598181,6.650575e-03,1.397948e-02


In [4]:
# add cluster specific metrics

res = []

for cluster, group in deg.groupby('cluster'):
    print(f"{cluster}...")

    new_frame = group.copy()

    # subset the clusters 
    cdf = adata[adata.obs['Nueron_Cluster'] == cluster]

    cluser_genes = new_frame['gene_symbol'].to_list()

    # get the read counts (raw)
    counts = cdf[:, cluser_genes].layers['counts']

    # get proportion pf cells expressing the gene
    props = counts.astype(bool).sum(axis=0) / cdf.shape[0]
    new_frame['perc_of_cluster_expressing'] = np.ravel(props)

    # get the mean read number within cluster
    read_counts = counts.mean(axis=0)
    new_frame['mean_cluster_reads'] = np.ravel(read_counts)

    # get mean expression
    tpm = cdf[:, cluser_genes].X
    mean_expression = tpm.mean(axis=0)
    new_frame['mean_cluster_TPM'] = np.ravel(mean_expression)
    
    res.append(new_frame)

res = pd.concat(res)
print(res.shape)

# outpath = "./neuron_subtype_deg.csv"
# res.to_csv(outpath, index=False)

res.head()

C1...
C2...
C3...
C4...
C5...
C6...
C7...
C8...
C9...
C10...
C11...
C12...
(67010, 9)


,cluster,gene_symbol,scores,log2foldchanges,pvals,pvals_benjamini-hochberg,perc_of_cluster_expressing,mean_cluster_reads,mean_cluster_TPM
23,C1,Dlx6os1,9.485873,0.944240,2.403620e-21,1.594355e-20,0.213830,0.395371,1.163099
73,C1,Meis2,3.491885,0.878733,4.796238e-04,1.100574e-03,0.077882,0.510670,0.511204
7,C1,Gad2,12.999899,0.695774,1.225052e-38,3.561462e-37,0.439024,1.045737,2.504741
2,C1,Nrxn3,25.884439,0.685811,9.970003e-148,2.776092e-144,0.891214,24.242212,6.989455
94,C1,Dlx1,2.713853,0.598181,6.650575e-03,1.397948e-02,0.083841,0.101718,0.428540


In [83]:
# plotly plots

alpha = 0.05

def make_volcano(df, outpath, show=False,):
    """A function to make an interactive volcano plot"""
    hoover_cols = [
        'names', 
        'logfoldchanges', 
        'pvals_adj', 
        'perc_of_cluster_expressing', 
        'mean_cluster_reads',
        'fasted_mean',
        'fed_mean',
    ]
    
    label_dict = {
        "logfoldchanges": "Fold Change (log2)",
        "logp": "p-value (-log10)",
        'fasted_mean' : 'TPM (mean fasted)',
        'fed_mean' : 'TPM (mean fed)',
        'perc_of_cluster_expressing' : 'Percent of Cells Expressing (within cluster)',
        'pvals_adj' : 'Adjusted p-value (Benjamini-Hochberg)',
        'names' : 'Gene Symbol',
        'mean_cluster_reads' : 'Mean Reads (within cluster)',
    }
    
    fig = px.scatter(df, 
                     x="logfoldchanges", 
                     y="logp",
                     size='perc_of_cluster_expressing',
                     color='fasted_mean',
                     hover_name="names",
                     hover_data=hoover_cols,
                     title=f"Cluster {cell_type}",
                     labels=label_dict,
                    )

    fig.add_vline(x=0.0)
    fig.add_vrect(x0=-1, x1=1, 
                  line_width=0, 
                  fillcolor="red",
                  opacity=0.2)
    
    fig.write_html(outpath)
    
    if show:
        fig.show()


res = []
for cell_type in sorted(adata.obs['Nueron_Cluster'].unique()):

    print(f"{cell_type}...")

    # subset the data by cell type
    cdf = adata[adata.obs['Nueron_Cluster'] == cell_type].copy()

    key = f'{cell_type}_by_diet'

    sc.tl.rank_genes_groups(cdf, 
                            groupby='Diet',
                            method='wilcoxon',
                            key_added=key,
                            corr_method='benjamini-hochberg')

    deg = sc.get.rank_genes_groups_df(cdf, 
                                      group='fasted',
                                      key=key)

    deg['cell_type'] = cell_type
    deg['logp'] = -np.log10(deg['pvals_adj'])

    print(f"{deg.shape=}")

    cluster_genes = deg['names'].to_list()

     # get the read counts (raw)
    counts = cdf[:, cluster_genes].layers['counts']

    # get proportion pf cells expressing the gene
    props = counts.astype(bool).sum(axis=0) / cdf.shape[0]
    deg['perc_of_cluster_expressing'] = np.ravel(props)

    # get the mean read number within cluster
    read_counts = counts.mean(axis=0)
    deg['mean_cluster_reads'] = np.ravel(read_counts)

    # get mean expression of groups
    mask = (cdf.obs['Diet'] == 'fasted')

    fasted_mean = cdf[mask, :].to_df().mean(axis=0)
    fed_mean = cdf[~mask, :].to_df().mean(axis=0)

    fasted_mean = fasted_mean.reindex(deg['names'])
    fed_mean = fed_mean.reindex(deg['names'])

    deg['fasted_mean'] = fasted_mean.values
    deg['fed_mean'] = fed_mean.values

    # filter out some genes
    deg = deg[~deg['names'].str.startswith('Gm')]
    deg = deg[~deg['names'].str.startswith('MT-')]
    deg = deg[~deg['names'].str.endswith('Rik')]
    deg = deg[deg['pvals_adj'] <= alpha]

    outpath = f"plotly/{cell_type}.html"
    make_volcano(deg, outpath)
    
    res.append(deg)

res = pd.concat(res)
print(f"{res.shape=}")
res.head()

C1...
deg.shape=(25060, 7)
C10...
deg.shape=(25060, 7)
C11...
deg.shape=(25060, 7)
C12...
deg.shape=(25060, 7)
C2...
deg.shape=(25060, 7)
C3...
deg.shape=(25060, 7)
C4...
deg.shape=(25060, 7)
C5...
deg.shape=(25060, 7)
C6...
deg.shape=(25060, 7)
C7...
deg.shape=(25060, 7)
C8...
deg.shape=(25060, 7)
C9...
deg.shape=(25060, 7)
res.shape=(11483, 11)


,names,scores,logfoldchanges,pvals,pvals_adj,cell_type,logp,perc_of_cluster_expressing,mean_cluster_reads,fasted_mean,fed_mean
0,Srsf10,22.204256,1.698258,3.124178e-109,3.914594e-105,C1,104.407313,0.729906,1.684675,4.747371,3.589521
1,Npy,16.634663,3.100751,3.909560e-62,1.959471e-58,C1,57.707861,0.187916,0.259285,1.613056,0.384164
3,Mt1,14.100804,1.845309,3.754653e-45,1.344166e-41,C1,40.871547,0.309174,0.436945,2.169625,1.150000
4,Ptbp2,13.751461,1.029376,4.991117e-43,1.389749e-39,C1,38.857064,0.704545,1.477199,4.318527,3.618793
5,Mblac2,13.387367,1.301093,7.167696e-41,1.632931e-37,C1,36.787032,0.490438,0.799750,3.046217,2.211657
